### AnuReddy , 3768482
### Smaran Nair , 3771609
### Yanxin Jia, 3769165
### Shreyansu Vyas, 3769429
### QUESTION 1
**1.1.1** 
-  Driver: since the main oncern is still to be decided based on the data obtained, this would be the typical text-driven problem where we derive conclusions and decisions stemming from the collected data itself.
-  Objective: the main objective of analyzing costumer data is to reach higher profit by better tailoring to specific customer needs. 
-  Data Availability: The scanned version of insurance paperworks would need further processing: conversion into raw text, cleaning, parsing etc. 
-  Cost factors: The cost of data gathering must be considered as necessary conversion&cleaning should be carried out, as well as the possible tranformation between formats. 

**1.1.2** 
-  Advantage:

Elasticsearch adapts the inverted index system of Lucene to offer speedy full-text searches, while traditional RDBMS based on SQL query could be relatively slower in response. The structuralized JSON documents where elasticsearch stores its data could be visited from any node instantly, reducing the number of read operations required.

Elasticsearch does not require user to specify the schema of its index; it performs well in auto-inference for datatypes such as number, boolean values and timestamps, offering certain flexibility in storage, while PostgreSQL operates on a rather rigid schema definition basis. 

-  Disadvantage:

The above can be also the drawback for elasticsearch, as PostgreSQL offers a plethora of functionalities based on its support of multiple data types.

Elasticsearch also sacrifices certain functionalities---such as transaction---for higher speed, while PostgreSQL supports a broader range of mechanisms including a robust transaction mechanism, while user can either roll back the operation to a given point or bundle operation together.

Elasticsearch has a more loose security system where every user is granted the same access as admin, while PostgreSQL tend to apply strict access control with multiple authentications for example LDAP etc.

Generally, based on the CAP theorem, what Elasticsearch offers is Availability and Partition Tolerance, while PostgreSQL offers Consistency and Availability. 

**1.1.3** 
**Stop word removal** 
-  Advantage: 
1. In certain scenarios where stop words are trivial (sentiment analysis, theme classification etc.), its removal would reduce corpus size greatly and keep the information provided by the corpus clean-cut and better fitting for classification or clustering. Example:

 "There are 5 people who are unsatisfied about the Mensa food today."->["unsatisfied" "food"]. 
 
The important keywords for sentiment are preserved while trivialities are cut down, word count decrease from 12 to 2. It lowers both the space dimension and data size greatly.
2. As the more significant words are preserved, the enhanced datasets with less noise should benefit training accuracy, training speed and model performance.

-  Disadvantage:
1. Inappropriate stop word removal could change the meaning of text drastically. For example if consider "not" as a stop word (as in Aruana) and remove it, "The food is not good" -> ["food" "good"], which is misleading. Sentiment analysis is sensitive to the stop words one select, and one should be careful of not leaving deciding features out. 

2. Object consisting of certain "stop words" would be unintelligible. example for search engine:

"to be or not to be, that is the question"->["question"]

"take that" ->[]

3. Problems arise because of loss of contexts due to stop word removal.\
Example as on slides:

["Jill" "reported" "CEO" "company"]<-"Jill reported to the CEO of the company" or "Jill reported as the CEO of the company".

If the model is not word-based but sequence-based, the training would encounter problems due to linguistic ambiguity. Thus stop word removal should not be considered if the preprocess is not specifically for word-based methods. 

**Stemming** 
-  Advantage: 
1. reduces size of datasets when using bag-of-words methods for assembling different forms of the same word together. For example: 

["liking" "likes" "liked"] -> ["like"]

2. increases processing speed. Stemming, as the purely algorithmic process, is often faster to use than applying word variants, since it does not require any further operation or context information.
-  Disadvantage:
1. Stemming certain words is not possible, as stemmers regularly can't recognize variants that deviate from the general grammar rules for example "good" and "better" and "best". ("So why not substitute it with "goodest"?"...Here is when the Orwellian Newspeak comes into use.)
2. Stemmed words can be ambiguous, for example Axes is plural for both Ax and Axis. This is called under-stemming, as roots and words are not always corresponding on a one-to-one basis.
3. There is also the phenomenon of over-stemming when words of very different meaning are stemmed into the same root. Basic example: while "the likes of" means "things similar to", "like" is also used as a verb. When "likes" as noun is stemmed and you are doing word-based sentiment analysis, it might be misleading. More example:  "farther" and "farthing" would be both stemmed into "farth" while their meanings differ greatly.

In [1]:
import pandas as pd
import os
import Levenshtein
import html
import PyPDF2
import datetime
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import sys
import fitz

### 2 Implement your own (basic) regular expression parser, which can accept or reject a RegEx statement.
### Consider the following specifications for your RegEx grammar that should be supported. You are not allowed to use any RegEx library, such as Python’s re or regex modules, for this task.

In [ ]:

def german_to_ascii(exp):
    if "ß" in exp:
        exp = exp.replace("ß", "")
    if "ä" in exp:
        exp = exp.replace("ä", "")
    if "ü" in exp:
        exp = exp.replace("ü", "")
    if "ö" in exp:
        exp = exp.replace("ö", "")
    return exp

In [ ]:
def verify_regex(exp, sequence=["^", "|", "*", "+", "?"]):
    #german letters
    stack = []
    exp = german_to_ascii(exp)
    
    #special chars sequence
    for r in range(len(exp) - 1):
        if exp[r] in sequence and exp[r+1] in sequence:
            return False


    for char in exp:
        if char in ["("]:
            stack.append(char)
        else:
            if not stack:
                return False
            current_char = stack.pop()
            if current_char == '(':
                if char != ")":
                    return False
            if current_char == '{':
                if char != "}":
                    return False
            if current_char == '[':
                if char != "]":
                    return False
    
    if stack:
        return False

    return all(ord(c) < 128 for c in exp)


In [ ]:
regex = "((ö)"
verify_regex(regex)

### Problem 1-3 Information Extraction from PDF Documents


In this assignment you have to convert a set of PDFs to raw text and extract information from the files using regular expressions.
1. Download the folders containing PDF files provided via Moodle (1-3-pdf-files.zip). Since the PDF format cannot be directly used to process text, you first have to convert the file contents to plain text. Find two different methods to convert PDFs to text, and compare their performance. You should provide a quantitative and qualitative analysis. For comparison of two generated files, use Python’s SequenceMatcher.ratio(). With this analysis as a basis, choose one of the methods, and provide the processed raw text files in the submission folder. Justify your decision!

### For the sake of comparsion of the methods i have used wegweiser_senioren.pdf from flyers document.

In [6]:
print(os.getcwd())

/Users/anureddy/Downloads/text-analysis/Assignment1


In [ ]:
input_file = open('Data01/1-3-pdf-files/flyers/1-3-pdf-files/flyers/wegweiser_senioren.pdf','rb')

## PyPDF2 text extraction 

In [4]:
now = datetime.datetime.now()
#file01 = open(r'Data01/1-3-pdf-files/flyers/output_file_pypdf1.txt','w')
with open('Data01/1-3-pdf-files/flyers/wegweiser_senioren.pdf','rb') as pdf_file:
    read_pdf = PyPDF2.PdfFileReader(pdf_file)
    number_of_pages = read_pdf.getNumPages()
    pypdf_test = ''
    for pg_no in range(number_of_pages): 
        page = read_pdf.getPage(pg_no)
        page_content = page.extractText()
        #file01.writelines(page_content)
        pypdf_test += page_content
        
        
#file01.close()        
stop = datetime.datetime.now()
pypdf_time = stop - now

# for the sake of comparison by the matrix iam using one page of the pdf

In [127]:
filesample01 =  open(r'Data01/1-3-pdf-files/flyers/output_file_pypdf_sample.txt','w')
fhandle = open(r'Data01/1-3-pdf-files/flyers/wegweiser_senioren.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(fhandle)
pagehandle = pdfReader.getPage(1)
page_content = pagehandle.extract_text()
filesample01.writelines(page_content)
filesample01.close()



## PyMuPDF text extraction

In [124]:
input_file1 = open('Data01/1-3-pdf-files/flyers/wegweiser_senioren.pdf')
now = datetime.datetime.now()
#file02 = open(r'Data01/1-3-pdf-files/flyers/output_file_pymupdf1.txt','w')

doc = fitz.open(input_file1)  
text = ''
for page in doc:  
    text += page.get_text() 

pymupdf_test = text
#file02.writelines(pymupdf_test) 
#file02.close()

stop = datetime.datetime.now()
pymupdf_time = stop - now

# for the sake of comparison by the matrix iam using one page of the pdf

In [130]:
file02 = open(r'Data01/1-3-pdf-files/flyers/output_file_pymupdf_sample.txt','w')
pg = doc.load_page(1)
pg1 = pg.get_text()
file02.writelines(pg1)
file02.close()


## Using NLP method parser from Tika Module 

In [112]:
from tika import parser
now = datetime.datetime.now()
file_data = parser.from_file('Data01/1-3-pdf-files/flyers/wegweiser_senioren.pdf')
 
# file_data['content'] is used to get the content of the pdf file.

output = file_data['content']

# This output.encode encodes the text into utf-8 format.

output1 = output.encode('utf-8', errors='ignore')
 
# finally it saves the text file to a file called output.txt.

#with open('Data01/1-3-pdf-files/flyers/nlp_tika_parser.txt', 'w') as the_file:
    #the_file.write(str(output1))
tika_time = stop - now

## Baseline text file (converted the given pdf to txt using online pdf converter)

In [103]:
now = datetime.datetime.now()

f = open('Data01/1-3-pdf-files/flyers/wegweiser_senioren.txt', "r", encoding="utf8")
baseline = f.read()

stop = datetime.datetime.now()
base_time = stop - now

## Cosine Similarity 

In [104]:
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

def clean_string(text):
    text = ''.join([word for word in text if word not in string.punctuation])
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text

corpi = [baseline,pypdf_test,pymupdf_test,output]

corpi = list(map(clean_string, corpi))

vectorizer = CountVectorizer().fit_transform(corpi)
vectors = vectorizer.toarray()

cosim = cosine_similarity(vectors)

def cosine_sim_vectors(vec1, vec2):
    vec1 = vec1.reshape(1, -1)
    vec2 = vec2.reshape(1, -1)
    return cosine_similarity(vec1, vec2)[0][0]

print("Cosine similarity for PypDF2:")
print(cosine_sim_vectors(vectors[0], vectors[1]))
print("Cosine similarity for MuPdf:")
print(cosine_sim_vectors(vectors[0], vectors[2]))
print("Cosine similarity for tika:")
print(cosine_sim_vectors(vectors[0], vectors[3]))

Cosine similarity for PypDF2:
0.9999616482818724
Cosine similarity for MuPdf:
0.9999666704679816
Cosine similarity for tika:
0.9998491826986143


## TDF-idf Similarity 

In [105]:
def tfidf_string_similarity_test(correct_string,test_string):
    import numpy as np
    import pandas as pd
    from sklearn.feature_extraction.text import TfidfVectorizer
    two_strings = [correct_string,test_string]
    vect = TfidfVectorizer(min_df=1, stop_words="english")
    tfidf = vect.fit_transform(two_strings)
    pairwise_similarity = tfidf * tfidf.T
    arr = pairwise_similarity.toarray()
    np.fill_diagonal(arr, -1)
    return arr[0][1]

In [106]:

print("Tfidf similarity for PyPDF2:")
print(tfidf_string_similarity_test(baseline, pypdf_test))
print("Tfidf similarity for PDFminer_six:")
print(tfidf_string_similarity_test(baseline, pymupdf_test))
print("Tfidf similarity for Tika:")
print(tfidf_string_similarity_test(baseline, output))

Tfidf similarity for PyPDF2:
0.9999711754667839
Tfidf similarity for PDFminer_six:
0.9999711754667839
Tfidf similarity for Tika:
0.9998676758240562


In [107]:
data = {'Time taken to extract the data ': [base_time, pypdf_time,pymupdf_time,tika_time],
        'Cosine similarity': [cosine_sim_vectors(vectors[0], vectors[0]), cosine_sim_vectors(vectors[0], vectors[1]), cosine_sim_vectors(vectors[0], vectors[2]), cosine_sim_vectors(vectors[0], vectors[3])],
        'Tf-idf similarity': [tfidf_string_similarity_test(baseline, baseline), tfidf_string_similarity_test(baseline, pypdf_test), tfidf_string_similarity_test(baseline, pymupdf_test),tfidf_string_similarity_test(baseline, output)]
        }
df = pd.DataFrame(data, index=['Baseline_string', 'PyPdf', 'PyMuPdf', 'Tika'])

In [108]:
df

,Time taken to extract the data,Cosine similarity,Tf-idf similarity
Baseline_string,0 days 00:00:00.001706,1.000000,1.000000
PyPdf,0 days 00:00:01.550275,0.999962,0.999971
PyMuPdf,0 days 00:00:00.105060,0.999967,0.999971
Tika,-1 days +23:59:50.500318,0.999849,0.999868


## comparing matrix Sequence Matcher 

In [ ]:
file1 = fitz.open('Data01/1-3-pdf-files/flyers/wegweiser_senioren.pdf')

In [168]:
input_file_pypdf = open('Data01/1-3-pdf-files/flyers/output_file_pypdf_sample.txt')
read_pypdf = input_file_pypdf.readlines()
input_file_mupypdf = open('Data01/1-3-pdf-files/flyers/output_file_pymupdf_sample.txt')
read_mupypdf = input_file_mupypdf.readlines()
baseline_file = open('Data01/1-3-pdf-files/flyers/Sample_plain_text')
read_baseline = baseline_file.readlines()

In [166]:
input_file_mupypdf = open('Data01/1-3-pdf-files/flyers/output_file_pymupdf_sample.txt', 'r')
read_mupypdf = [line.split(',') for line in input_file_mupypdf.readlines()]

In [176]:
read_mupypdf

['Grußwort \n',
 ' \n',
 'Liebe Mitbürgerinnen, \n',
 'liebe Mitbürger, \n',
 'das Wohl älterer Menschen ist mir ein wichtiges Anliegen, so dass ich mich persönlich dafür \n',
 'einsetze, das bestehende Angebot nachhaltig weiterzuentwickeln und zukunftsfähig zu halten. \n',
 'Ein wichtiger Schritt zur Orientierung über das breit gefächerte Angebot im Seniorenbereich \n',
 'ist eine aktualisierte Auflage des Wegweisers für ältere Menschen. Er enthält umfangreiche \n',
 'und altersspezifische Angebote, die ständig aktualisiert und durch neue ergänzt werden.  \n',
 'Auch im Internet unter www.heidelberg.de/senioren ist der Wegweiser zu finden. \n',
 'Die Angebote der Stadt Heidelberg sind vielfältig und abwechslungsreich. Eines von vielen \n',
 'Projekten sind die kostenlosen Bewegungstreffs im Freien, die in mehreren Stadtteilen \n',
 'angeboten werden. Das Netzwerk „Mehr Bewegung lebenslang“ wirbt mit dem Slogan „vital \n',
 'sein und bleiben“. Sich mit Gleichgesinnten zu treffen, gemei

In [175]:
read_pypdf

['Grußwort  Liebe Mitbürgerinnen, liebe Mitbürger, das Wohl älterer Menschen ist mir ein wichtiges Anliegen, so dass ich mich persönlich dafür einsetze, das bestehende Angebot nachhaltig weiterzuentwickeln und zukunftsfähig zu halten. Ein wichtiger Schritt zur Orientierung über das breit gefächerte Angebot im Seniorenbereich ist eine aktualisierte Auflage des Wegweisers für ältere Menschen. Er enthält umfangreiche und altersspezifische Angebote, die ständig aktualisiert und durch neue ergänzt werden.  Auch im Internet unter www.heidelberg.de/senioren ist der Wegweiser zu finden. Die Angebote der Stadt Heidelberg sind vielfältig und abwechslungsreich. Eines von vielen Projekten sind die kostenlosen Bewegungstreffs im Freien, die in mehreren Stadtteilen angeboten werden. Das Netzwerk „Mehr Bewegung lebenslang“ wirbt mit dem Slogan „vital sein und bleiben“. Sich mit Gleichgesinnten zu treffen, gemeinsame Ausflüge unternehmen oder sich sportlich zu betätigen. Wichtig ist es in Bewegung zu

In [173]:
from difflib import SequenceMatcher
sequenceScore = SequenceMatcher(None, read_pypdf, read_mupypdf)
print(sequenceScore.get_matching_blocks())
check_list = sequenceScore.ratio()*100
check_list = round(check_list,1) 
input_file_pypdf.close()
input_file_mupypdf.close()

[Match(a=1, b=34, size=0)]


In [174]:
check_list

0.0

In [171]:
from difflib import SequenceMatcher
sequenceScore = SequenceMatcher(None, read_baseline, read_mupypdf)
check_list = sequenceScore.ratio()*100
check_list = round(check_list,1) 
input_file_pypdf.close()
input_file_mupypdf.close()
baseline_file.close()


In [172]:
check_list

88.2

In [177]:
from difflib import SequenceMatcher
sequenceScore = SequenceMatcher(None, read_baseline, read_pypdf)
check_list = sequenceScore.ratio()*100
check_list = round(check_list,1) 
input_file_pypdf.close()
input_file_mupypdf.close()
baseline_file.close()

In [178]:
check_list

0.0

The pymupdf has done a decent job of converting the pdf to text file with the accuracy of 88% (same context and format) but since while onverting with pypdf its considering the entire page as block which is not as same as original file (format is different eventhough context is same), since sequencematcher does check the format and words should be in the same order as the original, it couldnt recognise the same context by pypdf method and hence the score is 0

3. The provided files are split into three groups. For each of the individual parts, make sure to print out all extracted results into a dedicated text file which should be included in your submission. Each line should contain only a single instance in a normalized format. For phone numbers, e.g., you could remove all spaces and special characters, etc.:

From the pdf provided, since each of them are in different format (some have images,tables only and paragraghs)
I am using 2 methods for each and apply for all the pdf:
1. MymuPdf to convert the paragraghs and images 
2. camelot to extract the tables from pdf 

### pdf from flyers folder 

In [84]:
file1 = fitz.open('Data01/1-3-pdf-files/flyers/bahnstadt.pdf')



In [85]:
# extract the text:
for pagenumber,page in enumerate(file1.pages(),start=1):
    text = page.get_text()
    txt = open(f'Data01/1-3-pdf-files/flyers/bahnstat/flyers_bahnstat_{pagenumber}.txt','a')
    txt.writelines(text)
    txt.close()


3. The provided files are split into three groups. For each of the individual parts, make sure to print out all extracted results into a dedicated text file which should be included in your submission. Each line should contain only a single instance in a normalized format. For phone numbers, e.g., you could remove all spaces and special characters, etc.:

(i) From the PDF files in the folder flyers/ extract as many valid phone numbers as possible.


In [14]:
import camelot
#file = bytes('Data01/1-3-pdf-files/flyers/bundeswehr.pdf','utf-8')
tables = camelot.read_pdf('Data01/1-3-pdf-files/flyers/bundeswehr.pdf',pages='all',flavor = 'lattice',strip_text ='\n')
tables[56].df#.to_csv('Data01/1-3-pdf-files/flyers/Tabular_text/bundeswehr.csv')

,0,1,2,3,4,5,6,7,8,9
0,Organisation,Liegenschaft,Anschrift,Ort,PLZ,E-Mail und URL,Telefon,Telefax,Telefon(IVBB),Telefax(IVBB)
1,Luftfahrt-Bundesamt,AußenstelleFrankfurt,Kelsterba-cher Straße23,Raunheim,65479,info@lba.dehttp://www.lba.de,+49 5312355-8207,+49 5312355-8299,,
2,Luftfahrt-Bundesamt,AußenstelleMünchen,Flughafen,München,85356,info@lba.dehttp://www.lba.de,+49 5312355-8410,+49 5312355-8499,,
3,Luftfahrt-Bundesamt,AußenstelleStuttgart,"Airport Busi-ness Center1, Gottlieb-Mainz-Stra...",Filderstadt,70794,info@lba.dehttp://www.lba.de,+49 5312355-8550,+49 5312355-8599,,
4,Luftfahrtamt der Bun-deswehr,Hauptadresse,Flughafen-straße 1,Köln-Wahn,51147,LufABwLeitungsbue-ro@bundeswehr.orghttp://www....,,,,
5,Luftwaffentruppen-kommando,Hauptadresse,Flughafen-straße 1,Köln,51147,LwTrKdoLdI@bundes-wehr.orghttp://www.luftwaffe.de,+49 2203908-4082,,,
6,Luftwaffenunterstüt-zungsgruppe Wahn,Hauptadresse,Flughafen-straße 1,Köln,51147,PIZLwCallCenter@bundeswehr.orghttp://www.luftw...,+49 2203908-3134,,,
7,Marinearsenal,Hauptadresse,KasinostraßeTor 7,Wilhelms-haven,26382,mars-posteingang@bundes-wehr.orghttp://www.baa...,+49 442149-0,+49 442149-2037,,
8,Marinearsenal,AußenstelleWarnemünde,Hohe Düne30,Rostock,18119,mars-posteingang@bundes-wehr.orghttp://www.baa...,+49 381636-3132,+49 381636-3108,,
9,Marinearsenal,Arsenalbe-trieb Kiel,KlausdorferWeg 2 - 24,Kiel,24148,mars-posteingang@bundes-wehr.orghttp://www.baa...,+49 431607-0,+49 431607-2585,,


In [15]:
df01 = pd.DataFrame()
for i in range(len(tables)):
    df01 = df01.append(tables[i].df,ignore_index = True)

/var/folders/w9/ksgfnxyn4jg66j5nhtx4v0mc0000gn/T/ipykernel_3463/3543817323.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df01 = df01.append(tables[i].df,ignore_index = True)


In [16]:
df02 = df01.rename(columns=df01.iloc[0]).drop(df01.index[0])

In [17]:
df02

,Organisation,Liegenschaft,Anschrift,Ort,PLZ,E-Mail und URL,Telefon,Telefax,Telefon(IVBB),Telefax(IVBB)
1,1. Panzerdivision,Hauptadresse,Bümmerste-der Tredde34,Oldenburg,26133,1.PzDivPressestelle@Bundeswehr.orghttp://www.d...,+49 441360-2150,+49 441360-2159,,
2,10. Panzerdivision,Hauptadresse,Oberdürrba-cher Straße,Veits-höchheim,97209,10.PzDivPressestel-le@bundeswehr.orghttp://www...,+49 9319707-3160,+49 9319707-3169,,
3,ABC-Abwehrkomman-do der Bundeswehr,Hauptadresse,Am Eichel-berg,Bruchsal,76646,ABCAbwKdoBwPres-se-undOeffentlich-keitsarbeit@...,+49 7251938 2500,+49 7251938 2500,,
4,Abschlussprüferauf-sichtsstelle,Hauptadresse,Uhlandstra-ße 88-90,Berlin,10717,infoapas@apasbafa.bund.dehttp://www.apasbafa.b...,+49 6196908-0,,,
5,Adalbert-Stifter-Vereine. V.,Hauptadresse,Hochstraße 8,München,81669,sekretariat@stifterver-ein.dehttp://www.stifte...,+49 89622716-30,+49 89 4891 148,,
...,...,...,...,...,...,...,...,...,...,...
1828,Zoologisches For-schungsmuseum Alex-ander Köni...,Hauptadresse,Adenaueral-lee 160,Bonn,53113,info@zfmk.dehttp://www.zfmk.de,+49 2289122-0,+49 2289122-212,,
1829,Zukunft-Umwelt-Ge-sellschaft gGmbH,Hauptadresse,KöthenerStraße 4,Berlin,10963,kontakt@z-u-g.orghttp://www.z-u-g.org,,,+49 307262-1431,
1830,Zukunftsforum Familiee. V.,Hauptadresse,Michaelkirch-str. 17/18,Berlin,10179,info@zukunftsforum-familie.dehttp://www.zukunf...,+49 30259272-820,+49 30259272-860,,
1831,Zusatzversorgungs-kasse für Arbeitneh-mer in d...,Hauptadresse,Druseltalstra-ße 51,Kassel,34131,info@zla.dehttp://www.zla.de,+49 56193279-0,+49 56193279-70,,


a) normalized format of telephone number 

In [18]:
df02.columns

Index(['Organisation', 'Liegenschaft', 'Anschrift', 'Ort', 'PLZ',
       'E-Mail und URL', 'Telefon', 'Telefax', 'Telefon(IVBB)',
       'Telefax(IVBB)'],
      dtype='object')

Extracted the phone numbers 

In [59]:
tp = df02.iloc[:,6:8]
tp

,Telefon,Telefax
1,+49 441360-2150,+49 441360-2159
2,+49 9319707-3160,+49 9319707-3169
3,+49 7251938 2500,+49 7251938 2500
4,+49 6196908-0,
5,+49 89622716-30,+49 89 4891 148
...,...,...
1828,+49 2289122-0,+49 2289122-212
1829,,
1830,+49 30259272-820,+49 30259272-860
1831,+49 56193279-0,+49 56193279-70


In [70]:
import re
s = ''
lst1 = []
#cols = ['Telefon','Telefax']
for i in df02['Telefon']:
    s = re.sub('[^0-9]+|"^Telefon"','', i)
    lst1.append(s)
lst1 = list(filter(None, lst1)) # to remove some of the empty stings from the list 

In [72]:
lst1

['494413602150',
 '4993197073160',
 '4972519382500',
 '4961969080',
 '498962271630',
 '4930200570',
 '4930200572000',
 '4930200573247',
 '4930200574000',
 '4930200571800',
 '49511348420',
 '49228249990',
 '492288330',
 '4947148310',
 '4947258190',
 '493312882100',
 '4946519560',
 '4971125930',
 '08002651050',
 '4942117610',
 '4964049242000',
 '4933127721111',
 '4923141930',
 '08001059000',
 '08004772000',
 '4921187910',
 '08002265726',
 '49308181990',
 '4922193713347',
 '4930185551855',
 '4930346460',
 '4951187010',
 '492289089930',
 '496196777260',
 '49221376800',
 '493023533671',
 '4922195710',
 '4951191150',
 '49514194831010',
 '493415950',
 '4997327840',
 '4951921298260',
 '49841886602110',
 '4975524052010',
 '4924156152113',
 '4930181702000',
 '4922899170',
 '496592172100',
 '494662820',
 '4950218000',
 '49228536831412',
 '4964517400',
 '4930184002030',
 '49301868111120',
 '4930186810',
 '49228993050',
 '493070300',
 '4930184413424',
 '49301868110920',
 '4930185809430',
 '49301817

In [57]:
with open('Data01/1-3-pdf-files/flyers/Qns3(1)/Phone_nos_result.txt', 'w') as f:
    for line in lst1:
        f.write(f"{line}\n")

In [81]:
import re
s = ''
lst1 = []
for i in df02['Telefax']:
    s = re.sub('[^0-9]+|"^Telefax"','', i)
    lst1.append(s)
lst1 = list(filter(None, lst1)) # to remove some of the empty stings from the list 

In [83]:
with open('Data01/1-3-pdf-files/flyers/Qns3(1)/Telefax_nos_result.txt', 'w') as f:
    for line in lst1:
        f.write(f"{line}\n")


### (ii) Extract valid URLs and Email addresses from the files in the folder flyers/.

In [3]:
file02 = fitz.open('Data01/1-3-pdf-files/flyers/bundeswehr.pdf')
for pagenumber,page in enumerate(file02.pages(),start=1):
    text = page.get_text()
    txt = open(f'Data01/1-3-pdf-files/flyers/Qns3(2)/bundeswehr_pdf_page_{pagenumber}.txt','a')
    txt.writelines(text)
    txt.close()

In [5]:
import glob

read_files = glob.glob("Data01/1-3-pdf-files/flyers/Qns3(2)/*.txt")

with open("Data01/1-3-pdf-files/flyers/Qns3(2)/result.txt", "wb") as outfile:
    for f in read_files:
        with open(f, "rb") as infile:
            outfile.write(infile.read())

In [6]:
with open('Data01/1-3-pdf-files/flyers/Qns3(2)/result.txt') as f:
    lines = f.readlines()

In [8]:
from re import match
pattern= '^(http|https://)[a-zA-Z0-9@:%._\\+~#?&//=]{2,256}?.de.*'
filtered_URL_values = list(filter(lambda v: match(pattern, v), lines))

print(filtered_URL_values)

['http://www.pei.de\n', 'http://www.bmwk.de\n', 'http://www.ptb.de\n', 'http://www.ptb.de\n', 'http://www.pbeakk.de\n', 'http://www.bundesre-\n', 'http://www.ptj.de\n', 'http://www.ptj.de\n', 'http://www.ptj.de\n', 'http://www.ptj.de\n', 'http://www.rki.de\n', 'http://www.rki.de\n', 'http://www.rki.de\n', 'http://www.rgzm.de\n', 'http://www.rosalux.de\n', 'http://www.bundestag.\n', 'http://www.vbg.de\n', 'http://www.vbg.de\n', 'http://www.vbg.de\n', 'http://www.vbg.de\n', 'http://www.vbg.de\n', 'http://www.vbg.de\n', 'http://www.vbg.de\n', 'http://www.vbg.de\n', 'http://www.vbg.de\n', 'http://www.vbg.de\n', 'http://www.vbg.de\n', 'http://www.vbg.de\n', 'http://www.unibw.de\n', 'http://www.luftwaffe.de\n', 'http://www.deutsches-\n', 'http://www.deutsches-\n', 'http://www.vebeg.de\n', 'http://www.vebeg.de\n', 'http://www.vzbv.de\n', 'http://www.agj.de\n', 'http://www.deutsches-\n', 'http://www.iud.bundes-\n', 'http://www.iud.bundes-\n', 'http://www.iud.bundes-\n', 'http://www.vbl.de\n', 

In [11]:
with open('Data01/1-3-pdf-files/flyers/Qns3(2)/URL_result.txt', 'w') as f:
    for line in filtered_URL_values:
        f.write(f"{line}")

In [10]:
pattern= '^[a-zA-Z0-9:%._\\+~#?&//=]+@[a-zA-Z0-9:%._\\+~#?&//=]+.de$'
filtered_Email_values = list(filter(lambda v: match(pattern, v), lines))

print(filtered_Email_values)

['havel@wsv.bund.de\n', 'havel@wsv.bund.de\n', 'havel@wsv.bund.de\n', 'ostsee@wsv.bund.de\n', 'ostsee@wsv.bund.de\n', 'ostsee@wsv.bund.de\n', 'ostsee@wsv.bund.de\n', 'ostsee@wsv.bund.de\n', 'rhein@wsv.bund.de\n', 'st.goar@wsv.bund.de\n', 'koblenz@wsv.bund.de\n', 'brohl@wsv.bund.de\n', 'rhein@wsv.bund.de\n', 'rhein@wsv.bund.de\n', 'passau@wsv.bund.de\n', 'passau@wsv.bund.de\n', 'neuses@wsv.bund.de\n', 'mdk@wsv.bund.de\n', 'elbe@wsv.bund.de\n', 'elbe@wsv.bund.de\n', 'elbe@wsv.bund.de\n', 'elbe@wsv.bund.de\n', 'elbe@wsv.bund.de\n', 'elbe@wsv.bund.de\n', 'niegripp@wsv.bund.de\n', 'wedel@wsv.bund.de\n', 'wedel@wsv.bund.de\n', 'wedel@wsv.bund.de\n', 'wedel@wsv.bund.de\n', 'rhein@wsv.bund.de\n', 'rhein@wsv.bund.de\n', 'havel@wsv.bund.de\n', 'havel@wsv.bund.de\n', 'havel@wsv.bund.de\n', 'havel@wsv.bund.de\n', 'havel@wsv.bund.de\n', 'havel@wsv.bund.de\n', 'weser@wsv.bund.de\n', 'weser@wsv.bund.de\n', 'weser@wsv.bund.de\n', 'weser@wsv.bund.de\n', 'weser@wsv.bund.de\n', 'weser@wsv.bund.de\n', 'we

In [12]:
with open('Data01/1-3-pdf-files/flyers/Qns3(2)/Email_result.txt', 'w') as f:
    for line in filtered_Email_values:
        f.write(f"{line}")

(iii) From the PDF file in the folder iban/ extract all IBANs.
Hint: You do not have to perform validation of the IBAN number, but only check for the format.

In [63]:
file02 = fitz.open('Data01/1-3-pdf-files/iban/liste1.pdf')
for pagenumber,page in enumerate(file02.pages(),start=1):
    text = page.get_text()
    txt = open(f'Data01/1-3-pdf-files/iban/iban_pdf_page_{pagenumber}.txt','a')
    txt.writelines(text)
    txt.close()

In [93]:
import fileinput
import glob

file_list = glob.glob("Data01/1-3-pdf-files/iban/*.txt")

with open('Data01/1-3-pdf-files/iban/result.txt', 'w') as file:
    input_lines = fileinput.input(file_list)
    file.writelines(input_lines)

In [47]:
with open('Data01/1-3-pdf-files/iban/result.txt') as f:
    lines = f.readlines()

In [48]:
from re import match
pattern = '^[A-Z]{2}[0-9]{2}(?:[ ]?[A-Z0-9]{2,4}){2,8}(?:[ ]?[A-Z0-9]){1,8}?$'
filtered_values_IBAN = list(filter(lambda v: match(pattern, v), lines))

print(filtered_values_IBAN)

['MR03 0002 0001 0100 0012 3456 753\n', 'MU07 BOMM 0101 1010 3030 0200 000M UR\n', 'ST68 0001 0001 0051 8453 1011 2\n', 'TN09 1420 7207 1007 0712 9648\n', 'BR00 0036 0305 0000 1000 9795 493P 1\n', 'VG06 VPVG 0000 0123 4567 8901\n', 'CR05 1520 2001 0262 8406 6\n', 'DO00 BAGR 0000 0001 2124 5361 1324\n', 'SV62 CENR 0000 0000 0000 0070 0025\n', 'GT82 TRAJ 0102 0000 0012 1002 9690\n', 'LC62 HEMM 0001 0001 0012 0012 0002 3015\n', 'AZ01 NABZ 0000 0000 1370 1000 1944\n', 'BH07 BMAG 0000 1299 1234 56\n', 'IL00 0108 0000 0009 9999 001\n', 'JO99 CITI 0010 0000 0000 0123 4567 89\n', 'QA58 DOHB 0000 1234 5678 90AB CDEF G\n', 'KW01 CBKU 0000 0000 0000 1234 5601 01\n', 'LB02 0999 0000 0001 0019 0122 9114\n', 'PK06 SCBL 0000 0011 2345 6702\n', 'PS02 PALS 0000 0000 0400 1234 5670 2\n', 'SA03 8000 0000 6080 1016 7519\n', 'TL38 0080 0123 4567 8910 157\n', 'TN09 1420 7207 1007 0712 9648\n', 'AE00 0331 2345 6789 0123 456\n', 'CY07 0020 0128 0000 0012 0052 7600\n', 'AL07 2121 1009 0000 0002 3569 8741\n', '

In [51]:
with open('Data01/1-3-pdf-files/iban/IBAN_nos.txt', 'w') as f:
    for line in filtered_values_IBAN:
        f.write(f"{line}")

4. Applyyoursolutionfromtask4-(i)tothefilesinthefolderscans/whichconsistsofpagesscannedfrom a phone book. Analyse how well your solution performs by giving examples.

In [92]:
from pathlib import Path
pdf_search = Path("Data01/1-3-pdf-files/scans").glob("*.pdf")
pdf_files = pdf_files = [str(file.absolute()) for file in pdf_search]


In [94]:
for pdf in pdf_files:
    with fitz.open(pdf) as doc:
       for pagenumber,page in enumerate(doc.pages(),start=1):
           text = page.get_text()
           txt = open(f'Data01/1-3-pdf-files/scans/ocr_page_{pagenumber}.txt','a')
           txt.writelines(text)
           txt.close()
           

In [97]:
with open('Data01/1-3-pdf-files/scans/ocr_page_1.txt') as f:
    lines = f.readlines()

In [98]:
lines

['Er \n',
 'www.dastelefonbuch.de \n',
 'FSEERRIL re IAIIIEISEE TE \n',
 'ne, \n',
 'nz \n',
 'Er \n',
 'EEE \n',
 '| $ \n',
 'WEEREuWENG \n',
 'Alt Peter \n',
 'Rechtsanwalt \n',
 'Fachanwalt für Arbeitsrecht \n',
 'Kirschgartenstr. 19 \n',
 '33836 70 \n',
 'www.kanzlei-buehler-alt.de \n',
 'm) \n',
 'Gratis anrufen | mit dastelefonbuch.de \n',
 '  \n',
 'Bau, Becky & Collegen \n',
 'Poststr. 2 \n',
 'www.bau-coll.de \n',
 'ed al Gratis anrufen \n',
 '| mit dastelefonbuch.de \n',
 'Becker \n',
 '& Kollegen \n',
 'Becker K \n',
 'en \n',
 'Rechtsanwälte \n',
 'Ralf Becker \n',
 'Fachanwalt für Bau- und Architektenrecht sowie Erbrecht \n',
 '= Immobilienrecht = Handels- und Gesellschaftsrecht \n',
 '| \n',
 'Vera Gräff \n',
 'Magisterin der Verwaltungswissenschaften \n',
 '| \n',
 '= öffentliches Recht = Arbeitsrecht = Familienrecht \n',
 'Louisa Hansen \n',
 'Do \n',
 'Arzthaftung / Medizinrecht = Kapitalanlagerecht \n',
 '| \n',
 '= Mietrecht = Versicherungsrecht \n',
 '| \n',
 'Tel. 